In [1]:
import pandas as pd, re
from bcp_reporte_mensual.pdf_processor import PDFprocessor

In [2]:
pdf = PDFprocessor('./pdf_sample/EECC_Dic2022_182.pdf')
df  = pdf.to_pandas()
n_pages = pdf.n_pages

## Content Data

In [3]:
df['is_date'] = df['line'].str.strip().str.match(r'^\d{2}-\d{2}').astype(int)
df['is_saldo'] = df['line'].str.strip().str.match(
    r'(\d+,)*\d+\.\d{2}-?$'
).astype(int)
group_page = df.groupby('page')
details_dict = dict()
saldo_dict = dict()
i_rel = []
for _n, group_df in group_page:
    # print(group_df)
    # if _n > 1:
    #     continue
    details_df = group_df.query('is_date == 1')
    i_rel.extend(details_df.index.values)

    max_details = details_df.nline.max()
    # print(max_details)
    saldo_f = group_df[(group_df['is_saldo'] == 1) & (group_df['nline'] > max_details)]
    # .query('is_saldo == 1 & nline > @max_details')['line'].values
    i_rel.extend(saldo_f.index.values)
    details_dict[_n]=details_df.line.values
    saldo_dict[_n] =saldo_f.line.values
details = []
saldos = []
for i in range(n_pages):
    details.extend(details_dict[i+1])
    saldos.extend(saldo_dict[i+1])


results = pd.DataFrame({
    "details": details,
    "saldos": saldos
})



# Aqui la parte inicial sobre todo para debugear

def split_date_description(value):
    match = re.match(r"(.*?\s)(VEN|CAJ|POS|TLC|INT|BPT|BPI)(\s.*)", value)
    if match:
        return match.groups()
    return value, '', ''  # En caso de que no haya coincidencia



results[['antes_codigo', 'codigo', 'despues_codigo']] = results['details'].apply(lambda x: pd.Series(split_date_description(x)))
results[['date', 'date_v', 'desc']] = results['antes_codigo'].apply(
    lambda x: pd.Series([x[:5], x[5:-23], x[-23:]])
)
results.drop(columns=['antes_codigo', 'details'], inplace=True)


### Extraer el tipo

def split_desc_tipo_value(value):
    match = re.match(r"(.*?)(\b\d{4}\b)(.*)", value)
    if match:
        return match.groups()
    return value, '', ''  # En caso de que no haya coincidencia

results[['ref', 'tipo', 'cargo']] = results['despues_codigo'].apply(lambda x: pd.Series(split_desc_tipo_value(x)))

results.drop(columns=['despues_codigo'], inplace=True)


### Extraer el suc-age

def split_desc_tipo_value(value):
    match = re.match(r"(.*?\s)(.{3}-.{3})(.*)", value)
    if match:
        return match.groups()
    return value, '', ''  # En caso de que no haya coincidencia

results[['lugar', 'suc_age', 'ref1']] = results['ref'].apply(lambda x: pd.Series(split_desc_tipo_value(x)))
# results[['date', 'date_v', 'desc']] = results['ref'].apply(
#     lambda x: pd.Series([x[:5], x[5:-23], x[-23:]])
# )
results.drop(columns=['ref'], inplace=True)

### Extraer el Hora

def split_hora(value):
    match = re.search(r"(\S+\s*)(\d{2}:\d{2})?(\s*\S*)", value)
    if match:
        return match.groups()
    return value, '', ''  # En caso de que no haya coincidencia

results[['num_op', 'hora', 'origen']] = results['ref1'].apply(lambda x: pd.Series(split_hora(x)))
# results[['date', 'date_v', 'desc']] = results['ref'].apply(
#     lambda x: pd.Series([x[:5], x[5:-23], x[-23:]])
# )
results.drop(columns=['ref1'], inplace=True)

flotantes = ['saldos', 'cargo']

orden = {
    "date": "FECHA PROC.",
    "date_v": "FECHA VALOR",
    "desc": "DESCRIPCION",
    "codigo": "MED AT*",
    "lugar": "LUGAR",
    "suc_age": "SUC-AGE",
    "num_op": "NUM OP",
    "hora": "HORA",
    "origen": "ORIGEN",
    "tipo": "TIPO",
    "cargo": "CARGO/ABONO",
    "saldos": "SALDO CONTABLE",
}
# orden.keys()

# orden = ['date', 'date_v', 'desc', '']
for columna in results.columns:
    results[columna] = results[columna].map(lambda x: x.strip() if isinstance(x, str) else x)

def convertir_a_flotante(cadena):
    # Eliminar las comas
    cadena_sin_comas = cadena.replace(",", "")
    cadena_sin_comas = cadena_sin_comas.replace("-", "")

    # Convertir a flotante
    try:
        numero = float(cadena_sin_comas)
    except ValueError:
        return None  # Devolver None si la conversión falla

    # Verificar si hay un signo negativo
    if "-" in cadena:
        numero = -numero

    return numero

for numeric_col in flotantes:
    results[numeric_col] = results[numeric_col].apply(convertir_a_flotante)

results = results.replace("", None)
results_orden = results[orden.keys()]


In [ ]:
df1 = results

sorted_df = df1.copy().reset_index()


sorted_df= sorted_df.sort_values(['date', 'index'])


sorted_df = sorted_df.reset_index(drop=True)
sorted_df = sorted_df[orden.keys()]
sorted_df = sorted_df.rename(columns=orden)
# output sorted_df


In [9]:
sorted_df

,FECHA PROC.,FECHA VALOR,DESCRIPCION,MED AT*,LUGAR,SUC-AGE,NUM OP,HORA,ORIGEN,TIPO,CARGO/ABONO,SALDO CONTABLE
0,01-12,None,EFECTIVO00000009910688,VEN,None,111-023,016321,07:40,BMOA20,1013,663.54,64973.42
1,01-12,None,EFECTIVO00000002892009,VEN,None,111-023,787847,13:00,BMO892,1013,924.40,65897.82
2,01-12,None,COMIS.RECAUDACION,INT,None,194-000,896379,None,None,4983,-2.40,65895.42
3,02-12,None,EFECTIVO00000070245924,VEN,None,111-023,150857,17:59,BMO522,1013,478.33,66373.75
4,02-12,None,EFECTIVO00000007506372,VEN,None,111-023,177082,10:57,BMO533,1013,674.35,67048.10
...,...,...,...,...,...,...,...,...,...,...,...,...
124,31-12,None,EFECTIVO00000043974260,VEN,AG.ARNALDO MARQUE,193-017,173283,11:11,S95362,1013,666.66,132118.19
125,31-12,None,EFECTIVO00000043974260,VEN,AG.ARNALDO MARQUE,193-017,179647,11:13,S95362,1013,666.66,132784.85
126,31-12,None,ENVIO.EST.CTA ¥,INT,None,194-000,103994,None,None,4991,-1.32,132783.53
127,31-12,None,COMIS.RECAUDACION,INT,None,194-000,132194,None,None,4983,-5.70,132777.83


## Detalles

In [6]:
# details

details = df[~df.index.isin(i_rel)]
details
details['clean_line'] = details['line'].apply(lambda x: str(x).strip().lower())
detalles_repetidos = details.value_counts('clean_line').reset_index().query('count == @n_pages')

def detect_num_cuenta(text):
    patron_num = r'\d+(?:-\d+)+'
    match = re.findall(patron_num, text)
    if match:
        return match[0]
    return None

def detect_type_cuenta(text):
    if pd.isna(text) or not isinstance(text, str):
        return None
    
    # Convert to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Check if contains "cuenta"
    if "cuenta" in text_lower:
        # Find the position of "cuenta"
        cuenta_pos = text_lower.find("cuenta")
        
        # Extract everything after "cuenta" plus its length (6 characters)
        after_cuenta = text[(cuenta_pos + 6):].strip()
        if len(after_cuenta.strip()) == 0:
            return None 
        
        return after_cuenta.capitalize()
    
    return None

def detect_dir_cuenta(text):
    if pd.isna(text) or not isinstance(text, str):
        return None
    
    # Convert to lowercase for case-insensitive matching
    text_lower = text.lower()
    
    # Check for account number pattern
    # Check if starts with "pagi"
    if "primavera" in text:
        return 1
    
    return None

def detect_moneda(text):
    monedas = {
        "soles": "Soles",
        "dolares": "Dólares"
    }
    if text in monedas.keys():
        return monedas[text]
    return None



C:\Users\JhonK\AppData\Local\Temp\ipykernel_11892\3641648762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details['clean_line'] = details['line'].apply(lambda x: str(x).strip().lower())


In [7]:
range_details = details[
    (details['clean_line'].isin(detalles_repetidos['clean_line']))  
].query('page == 1')

cols_details_cuenta = {'acc_num': "Numero de cuenta", 'type_acc': "Tipo de cuenta", 'moneda': "Moneda"}



range_details['acc_num'] = range_details['clean_line'].apply(detect_num_cuenta)
range_details['type_acc'] = range_details['clean_line'].apply(detect_type_cuenta)
range_details['moneda'] = range_details['clean_line'].apply(detect_moneda)
range_details['direccion'] = range_details['clean_line'].apply(detect_dir_cuenta)
i_direccion_ubicacion = range_details.dropna(subset = 'direccion').nline.values[0]
name_company = range_details[range_details.nline == i_direccion_ubicacion - 1][['line']].to_dict('records')
info_details = range_details.dropna(subset=cols_details_cuenta, how='all')

info_ref = info_details[cols_details_cuenta.keys()]
info_ref['acc_name'] = name_company[0].get('line')
# info_ref
details_results = info_ref.melt(var_name='variable', value_name='valor').dropna().drop_duplicates()
details_results

C:\Users\JhonK\AppData\Local\Temp\ipykernel_11892\4292035843.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_ref['acc_name'] = name_company[0].get('line')


,variable,valor
1,acc_num,194-9734410-1-82
5,type_acc,Corriente
6,moneda,Dólares
9,acc_name,DISTRIBUIDORA ASIA PERU SUR GRANDE S.A.C ...


In [8]:
details_results.to_dict()

{'variable': {1: 'acc_num', 5: 'type_acc', 6: 'moneda', 9: 'acc_name'},
 'valor': {1: '194-9734410-1-82',
  5: 'Corriente',
  6: 'Dólares',
  9: 'DISTRIBUIDORA ASIA PERU SUR GRANDE S.A.C               '}}